In [1]:
%load_ext autoreload
%autoreload 2

from api_tricount import *


In [2]:

data, dict = fetch_data()
data = build_data(data)
detail = split_data(data, dict)
postes, result = concat_data(detail, dict)



    # --------------------------------- #
    # Connection to Google Sheet        #
    # and extract Spreadsheets          #
    #  - exported tricount data         #
    #  - category dictonnary            #
    # --------------------------------- #
    
 -- data fetched
(1023, 17)
Index(['Unnamed: 0', 'Titre', 'Montant', 'Devise', 'Taux de change',
       'Montant dans la devise du tricount (EUR)', 'Type de transaction',
       'Catégorie', 'Payé par', 'Payé par Commun', 'Payé par Lucie',
       'Payé par Vincent', 'Impacté à Commun', 'Impacté à Lucie',
       'Impacté à Vincent', 'Date & heure', 'URL des images'],
      dtype='object')
 -- dict fetched 
{'Alimentation': 'Quotidien', 'Shopping': 'Achats', 'Logement': 'Achats', 'Loyer & charges': 'Quotidien', 'Divertissement': 'Extra', 'Sport': 'Loisir', 'Restaurant & bar': 'Extra', 'Investissement': 'Investissement', 'Transport': 'Quotidien', 'Week-end': 'Loisir', 'Soin de santé': 'Quotidien', 'Livres': 'Achats', 'Activités': 'Lois

In [83]:
%load_ext autoreload
%autoreload 2
from api_portfolio import build_data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
data = build_data()
s = data.iloc[-1]


    # ----------------------------------------------
    # Get data, concate, merge and extract metrics
    # for portfolio Dashboard 
    # ----------------------------------------------
    
              Market Currency     Forex IsDepot  PriceFmt AmountFmt  ValueFmt
Asset                                                                        
€           EUREUR=X        €  EUREUR=X    TRUE  {:.0f} €    {:.0f}  {:.0f} €
$              EUR=X        €  EUREUR=X    TRUE  {:.4f} €    {:.0f}  {:.0f} €
USDT           EUR=X        €  EUREUR=X    TRUE  {:.4f} €    {:.0f}  {:.0f} €
ESE           ESE.PA        €  EUREUR=X          {:.2f} €    {:.0f}  {:.2f} €
PUST                        €  EUREUR=X          {:.2f} €    {:.0f}  {:.2f} €
RS2K                        €  EUREUR=X          {:.2f} €    {:.0f}  {:.2f} €
OBLI         OBLI.PA        €  EUREUR=X          {:.2f} €    {:.0f}  {:.0f} €
GOLD         GOLD.PA        €  EUREUR=X          {:.2f} €    {:.0f}  {:.0f} €
BTC          BTC-USD     U

In [85]:
# s['PnL', 'ZEN', '$']
s[:, 'ZEN', '$'].index


Index(['Amount', 'Deposit', 'DepositEUR', 'Cotation', 'IsDepot', 'Currency',
       'PriceFmt', 'AmountFmt', 'ValueFmt'],
      dtype='object')

In [14]:
np.unique(assets['Currency'])

array(['$', 'USDT', '€'], dtype=object)

In [27]:
assets['USDT']['Market']

KeyError: 'USDT'

In [16]:
import streamlit as st
import gspread as gs 
import pandas as pd
import numpy as np
import yfinance as yf

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


# gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
# ss = gc.open_by_key(st.secrets['portfolio'].spreadsheet_key)
# dicts = pd.DataFrame(ss.worksheet('Dict').get_all_records())
# operation = pd.DataFrame(ss.worksheet('Operations').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
# greenbull = pd.DataFrame(ss.worksheet('GREENBULL').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
# assets = dicts.set_index('Asset')
# print(assets, '\n')

# market = yf.download(' '.join([*assets['Forex'], *assets['Market'][:-1]]), start='2021-04-28')['Close']
# # print(market.columns)
# market = pd.concat([market, greenbull], axis=1)
# market = pd.concat([market], keys=['Market'], axis=1)
# market = pd.concat([market], keys=['Cotation'], axis=1)

operation: pd.DataFrame = operation
assets: pd.DataFrame = assets
market: pd.DataFrame = market

df = market.ffill().bfill()
# print(df.head())

for portfolio in np.unique(operation['Portfolio']):
    print(portfolio)
    dfp: pd.DataFrame = operation[operation['Portfolio'] == portfolio]

    print(f"\tAssets")
    for asset in np.unique(dfp['Asset']):
        print(f"\t\t{assets.loc[asset]['Class']}:\t{asset}")

        dfa: pd.DataFrame = dfp[dfp['Asset'] == asset]
        dfa = dfa.groupby('Date').agg({'Quantity': "sum", 'Total': "sum"}).reindex(df.index).fillna(0)

        df['Quantity', portfolio, asset] = dfa['Quantity']
        df['Total', portfolio, asset] = dfa['Total']
    
    print(f"\tCurrency")
    for currency in np.unique(dfp['Currency']):
        print(f"\t\t{assets.loc[currency]['Class']}:\t{currency}")

        dfc: pd.DataFrame = dfp[dfp['Currency'] == currency]
        dfc = dfc.groupby('Date').agg({'Quantity': "sum", 'Total': "sum"}).reindex(df.index).fillna(0)
        
        if ('Quantity', portfolio, currency) not in df.columns:
            df['Quantity', portfolio, currency] = 0
        if ('Total', portfolio, currency) not in df.columns:
            df['Total', portfolio, currency] = 0

        df['Quantity', portfolio, currency] -= dfc['Total']
        df['Total', portfolio, currency] -= dfc['Total'] * df['Cotation', 'Market', assets.loc[currency]['Market']]


    print(f"\tBuild datatable ...")
    for asset in np.unique([*dfp['Asset'], *dfp['Currency']]):
        print("\t", asset)

        amount, invest = pd.Series(dtype=float), pd.Series(dtype=float)
        for idx in df.index:
            amount[idx] = np.sum(df.loc[:idx]['Quantity', portfolio, asset])
            invest[idx] = np.sum(df.loc[:idx]['Total', portfolio, asset])

        df = pd.concat([df, pd.DataFrame({
                ('Amount', portfolio, asset): amount,
                ('Invested', portfolio, asset): invest,
            })], axis=1)

        df['Class', portfolio, asset] = assets.loc[asset]['Class']
        df['PriceFmt', portfolio, asset] = assets.loc[asset]['PriceFmt']
        df['ValueFmt', portfolio, asset] = assets.loc[asset]['ValueFmt']
        df['AmountFmt', portfolio, asset] = assets.loc[asset]['AmountFmt']

        if assets.loc[asset]['Class'] == 'Deposit':
            df['Deposite', portfolio, asset] = df['Invested', portfolio, asset]
            df['DepositeEUR', portfolio, asset] = df['Deposite', portfolio, asset]
        
        else:
            df['PRU', portfolio, asset] = df['Invested', portfolio, asset] / df['Amount', portfolio, asset]
            df['Value', portfolio, asset] = df['Amount', portfolio, asset] * df['Cotation', 'Market', assets.loc[asset]['Market']]
            df['PnL', portfolio, asset] = df['Value', portfolio, asset] - df['Invested', portfolio, asset]
            
            toEUR = df['Cotation', 'Market', assets.loc[asset]['Forex']]
            df['InvestedEUR', portfolio, asset] = df['Invested', portfolio, asset] / toEUR
            df['ValueEUR', portfolio, asset] = df['Value', portfolio, asset] / toEUR
            df['PnLEUR', portfolio, asset] = df['PnL', portfolio, asset] / toEUR


    print(f"\tConcate data ...")
    df['DepositeEUR', portfolio, 'All'] = df.loc[:, pd.IndexSlice['DepositeEUR', portfolio, :]].sum(axis=1)
    df['InvestedEUR', portfolio, 'All'] = df.loc[:, pd.IndexSlice['InvestedEUR', portfolio, :]].sum(axis=1)
    df['ValueEUR', portfolio, 'All'] = df.loc[:, pd.IndexSlice['ValueEUR', portfolio, :]].sum(axis=1)
    df['PnLEUR', portfolio, 'All'] = df.loc[:, pd.IndexSlice['PnLEUR', portfolio, :]].sum(axis=1)
    df['CashEUR', portfolio, 'All'] = df['DepositeEUR', portfolio, 'All'] - df['InvestedEUR', portfolio, 'All']

df['DepositeEUR', 'All', 'All'] = df.loc[:, pd.IndexSlice['DepositeEUR', :, 'All']].sum(axis=1)
df['InvestedEUR', 'All', 'All'] = df.loc[:, pd.IndexSlice['InvestedEUR', :, 'All']].sum(axis=1)
df['ValueEUR', 'All', 'All'] = df.loc[:, pd.IndexSlice['ValueEUR', :, 'All']].sum(axis=1)
df['PnLEUR', 'All', 'All'] = df.loc[:, pd.IndexSlice['PnLEUR', :, 'All']].sum(axis=1)
df['CashEUR', 'All', 'All'] = df.loc[:, pd.IndexSlice['CashEUR', :, 'All']].sum(axis=1)


for column in df.columns:
    print(column)
# dates = ["2021-04-30", "2021-04-30", "2021-07-30", "2021-07-30", "2021-08-10", "2021-08-10", "2021-10-01", "2021-10-01", "2021-11-01", "2021-11-01", "2021-12-01", "2021-12-01", "2021-12-31", "2022-02-01", "2022-02-01", "2022-03-24", "2022-03-24", "2022-04-01", "2022-04-01", "2022-04-01", "2022-04-05", "2022-04-05", "2022-04-13", "2022-04-13", "2022-05-02", "2022-05-02", "2022-04-19", "2022-04-19", "2022-06-01", "2022-06-01", "2022-06-01", "2022-06-01", "2022-07-01", "2022-07-04", "2022-07-05", "2022-07-05", "2022-08-05", "2022-08-05", "2022-08-25", "2022-10-12", "2022-10-12", "2022-10-12", "2023-02-10"]
dates = ["2021-04-30","2021-07-30","2021-08-10","2021-10-01","2021-11-01","2021-12-01","2021-12-31","2022-02-01","2022-03-24","2022-04-01","2022-04-05","2022-04-13","2022-05-02","2022-04-19","2022-06-01","2022-07-01","2022-07-04","2022-07-05","2022-08-05","2022-08-25","2022-10-12","2023-02-10"]
# df.loc[dates]
# df.head()
df.loc[dates]   #, pd.IndexSlice[['Quantity', 'Amount', 'Total', 'Invested'], 'ZEN', ['BTC', 'USDT', 'EUR']]]


DMA
	Assets
		Cash:	€
	Currency
		Deposit:	EUR
	Build datatable ...
	 EUR
	 €
	Concate data ...
ZEN
	Assets
		Cash:	$
		Crypto:	BTC
		Stocks:	ESE
		Commo:	GOLD
		Stocks:	GREENBULL
		Bonds:	OBLI
		Cash:	USDT
		Cash:	€
	Currency
		Cash:	$
		Deposit:	EUR
		Cash:	USDT
		Cash:	€
	Build datatable ...
	 $
	 BTC
	 ESE
	 EUR
	 GOLD
	 GREENBULL
	 OBLI
	 USDT
	 €
	Concate data ...
('Cotation', 'Market', 'BTC-USD')
('Cotation', 'Market', 'ESE.PA')
('Cotation', 'Market', 'EUR=X')
('Cotation', 'Market', 'EUREUR=X')
('Cotation', 'Market', 'EURUSD=X')
('Cotation', 'Market', 'GOLD.PA')
('Cotation', 'Market', 'OBLI.PA')
('Cotation', 'Market', 'GREENBULL')
('Quantity', 'DMA', '€')
('Total', 'DMA', '€')
('Quantity', 'DMA', 'EUR')
('Total', 'DMA', 'EUR')
('Amount', 'DMA', 'EUR')
('Invested', 'DMA', 'EUR')
('Class', 'DMA', 'EUR')
('PriceFmt', 'DMA', 'EUR')
('ValueFmt', 'DMA', 'EUR')
('AmountFmt', 'DMA', 'EUR')
('Deposite', 'DMA', 'EUR')
('DepositeEUR', 'DMA', 'EUR')
('Amount', 'DMA', '€')
('Invested', 'DMA'

Cotation                                                     \
                  Market                                                      
                 BTC-USD     ESE.PA     EUR=X EUREUR=X  EURUSD=X    GOLD.PA   
2021-04-30  57750.175781  16.617201  0.824600      1.0  1.212709  58.556999   
2021-07-30  42235.546875  17.775999  0.840831      1.0  1.189300  61.261002   
2021-08-10  45585.031250  18.164900  0.851900      1.0  1.173847  58.730000   
2021-10-01  48116.941406  17.935900  0.863970      1.0  1.157448  60.480000   
2021-11-01  61004.406250  19.136600  0.865300      1.0  1.155668  61.630001   
2021-12-01  57229.828125  19.755699  0.882590      1.0  1.133029  62.779999   
2021-12-31  46306.445312  20.323900  0.883000      1.0  1.132503  63.974998   
2022-02-01  38743.273438  19.362900  0.890500      1.0  1.122965  64.012001   
2022-03-24  43960.933594  19.706200  0.908570      1.0  1.100631  71.120003   
2022-04-01  46281.644531  19.841999  0.903150      1.0  1.107236  69.559998   
2022-04-05  45555.992188  20.206400  0.911090      1.0  1.097586  70.213997   
2022-04-13  41166.730469  19.721800  0.923690      1.0  1.082614  72.510002   
2022-05-02  38529.328125  18.980000  0.949130      1.0  1.053596  70.620003   
2022-04-19  41502.750000  19.922300  0.927490      1.0  1.078179  72.150002   
2022-06-01  29799.080078  18.601299  0.931610      1.0  1.073411  69.028000   
2022-07-01  19269.367188  17.629601  0.954410      1.0  1.047768  69.099998   
2022-07-04  20231.261719  17.734100  0.958410      1.0  1.043395  68.974998   
2022-07-05  20190.115234  17.765200  0.958480      1.0  1.043319  68.634003   
2022-08-05  23289.314453  19.699699  0.975820      1.0  1.024779  69.716003   
2022-08-25  21600.904297  20.290501  1.003100      1.0  0.996910  70.130997   
2022-10-12  19157.445312  18.033899  1.030910      1.0  0.970017  68.440002   
2023-02-10  21651.183594  18.693300  0.930900      1.0  1.074229  69.239998   

                             Quantity   Total  ... DepositeEUR   InvestedEUR  \
                                  DMA     DMA  ...         ZEN           ZEN   
           OBLI.PA GREENBULL        €       €  ...         All           All   
2021-04-30  10.786   14140.0      0.0     0.0  ...      -500.0    500.000013   
2021-07-30  11.030   15220.0      0.0     0.0  ...     -1000.0   1009.706102   
2021-08-10  11.080   15140.0    100.0   100.0  ...     -1300.0   1322.800765   
2021-10-01  10.844   15060.0      0.0     0.0  ...     -1800.0   1837.079634   
2021-11-01  10.756   15810.0      0.0     0.0  ...     -2300.0   2339.417847   
2021-12-01  10.920   14920.0      0.0     0.0  ...     -3300.0   3379.755188   
2021-12-31  10.758   14920.0      0.0     0.0  ...     -3300.0   3380.957941   
2022-02-01  10.608   14130.0      0.0     0.0  ...     -3800.0   3906.705356   
2022-03-24  10.182   13080.0   8400.0  8400.0  ...     -5000.0   5178.768564   
2022-04-01  10.078   13850.0      0.0     0.0  ...     -7500.0   7657.153513   
2022-04-05   9.989   13850.0      0.0     0.0  ...     -7500.0   7693.026497   
2022-04-13   9.947   13850.0      0.0     0.0  ...     -7500.0   7752.725213   
2022-05-02   9.759   12460.0      0.0     0.0  ...     -9500.0   9878.856547   
2022-04-19   9.846   13850.0      0.0     0.0  ...     -9500.0   9771.565539   
2022-06-01   9.541   12270.0      0.0     0.0  ...    -10000.0  10288.138392   
2022-07-01   9.539   11380.0      0.0     0.0  ...    -10000.0  10422.840449   
2022-07-04   9.466   11380.0      0.0     0.0  ...    -10000.0  10447.352519   
2022-07-05   9.516   11380.0      0.0     0.0  ...    -11500.0  11947.781329   
2022-08-05   9.780   12090.0      0.0     0.0  ...    -11500.0  12054.040837   
2022-08-25   9.417   12090.0      0.0     0.0  ...    -12000.0  12727.214659   
2022-10-12   8.739   10900.0      0.0     0.0  ...    -12000.0  12903.752424   
2023-02-10   8.970   12915.0      0.0     0.0  ...    -12000.0  12246.902720   

                ValueEUR       PnLEUR  

In [37]:
('Ctation', 'Market',   'BTC-USD') in df.columns

False